# upload dataset and import pytorch 


In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving new_data.csv.zip to new_data.csv.zip
User uploaded file "new_data.csv.zip" with length 117841345 bytes


In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch


tcmalloc: large alloc 1073750016 bytes == 0x583a2000 @  0x7f86a097b2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [4]:

! unzip new_data.csv.zip

Archive:  new_data.csv.zip
  inflating: new_data.csv            
   creating: __MACOSX/
  inflating: __MACOSX/._new_data.csv  


In [1]:
!ls


sample_data


In [41]:
import torch
import pandas as pd
df = pd.read_csv("new_data.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,31,40,43,44,45,48,49,61,68,69,70,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

df1 = df.loc[df['totals_transactionRevenue'] == 0]
dfx = df1.sample(20000)
df2 = df.loc[df['y'] > 0]
data = pd.concat([dfx,df2])

In [0]:
# len(df2)

18514

# add column: year month weekday

In [0]:
import pandas as pd
# data = pd.read_csv("new_data.csv")
df = data
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# get new year feature
df['year'] = df['date'].dt.year
# get new month feature
df['month'] = df['date'].dt.month
# get new week feature 
df['week'] = df['date'].dt.week
# get new weekday feature 
df['weekday'] = df['date'].dt.weekday
# drop 'date'
del(df['date'])


In [0]:
# import math
# df = data["totals_transactionRevenue"].apply(lambda x: math.log(1 + x))
# df
# data["totals_transactionRevenue"]
# max(data)
# df['visitStartTime']
# realtime = pd.to_datetime(df['visitStartTime'], unit='s')
# real = pd.to_datetime(realtime, format='%Y%m%d%H:%M:%s')
# df['weekday']

In [0]:
# real.dt.second
realtime = pd.to_datetime(df['visitStartTime'], unit='s')
real = pd.to_datetime(realtime, format='%Y%m%d%H:%M:%s')
df['hour'] = real.dt.hour
df['minute'] = real.dt.minute
df['second'] = real.dt.second

In [0]:
import numpy as np
import pandas
# data = pandas.read_csv("new_data.csv")
data = df
data = data.replace(np.nan,0)
str_cols = ["channelGrouping", "device_browser", 
            "device_deviceCategory", "device_operatingSystem", 
            "geoNetwork_city", "geoNetwork_continent", 
            "geoNetwork_country", "geoNetwork_metro",
            "geoNetwork_networkDomain", "geoNetwork_region", 
            "geoNetwork_subContinent", "customDimensions_value"]
# str_cols = ["channelGrouping", 
#             "geoNetwork_city", "geoNetwork_continent", 
#             "geoNetwork_country", "geoNetwork_metro",
#             "geoNetwork_networkDomain", "geoNetwork_region", 
#             "geoNetwork_subContinent", "customDimensions_value"]
data[str_cols] = str(data[str_cols])
num_cols = ["year", "month","weekday","hour","minute","second","totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits',
            'totals_timeOnSite','totals_transactions'] 
# num_cols = ["totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits'] 


import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder


# data["totals_transactionRevenue"] = data["totals_transactionRevenue"].apply(lambda x: math.log(1 + x))
# data["totals_transactionRevenue"] = data["y"]
    
label_encoders = {}
for str_col in str_cols:
    label_encoders[str_col] = LabelEncoder()
    data[str_col] = label_encoders[str_col].fit_transform(data[str_col].astype(str))

# training_data = df
msk = numpy.random.rand(len(data)) < 0.8
training_data = data[msk]
testing_data = data[~msk]
testing_id = testing_data['fullVisitorId']
target = training_data['totals_transactionRevenue']
training_data = training_data[str_cols + num_cols]   


#create the optimizer

import torch.utils.data as data_utils
training_data = training_data.values.astype('float')
training_data = torch.cuda.FloatTensor(training_data)
target = torch.cuda.FloatTensor(target.values)

loss_list = []


training_data = training_data.unsqueeze(0)
training_data = training_data.permute(1,0,2)
target = target.unsqueeze(0)
target = target.permute(1,0)
training_data = training_data.cuda()
target = target.cuda()
train = data_utils.TensorDataset(training_data, target)


In [23]:
list(df)

['Unnamed: 0',
 'channelGrouping',
 'fullVisitorId',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'device_browser',
 'device_deviceCategory',
 'device_operatingSystem',
 'geoNetwork_city',
 'geoNetwork_continent',
 'geoNetwork_country',
 'geoNetwork_metro',
 'geoNetwork_networkDomain',
 'geoNetwork_region',
 'geoNetwork_subContinent',
 'totals_bounces',
 'totals_hits',
 'totals_newVisits',
 'totals_pageviews',
 'totals_sessionQualityDim',
 'totals_timeOnSite',
 'totals_transactionRevenue',
 'totals_transactions',
 'trafficSource_adContent',
 'trafficSource_adwordsClickInfo.adNetworkType',
 'trafficSource_adwordsClickInfo.gclId',
 'trafficSource_adwordsClickInfo.isVideoAd',
 'trafficSource_adwordsClickInfo.page',
 'trafficSource_adwordsClickInfo.slot',
 'trafficSource_isTrueDirect',
 'trafficSource_keyword',
 'trafficSource_medium',
 'trafficSource_referralPath',
 'trafficSource_source',
 'hits_appInfo.exitScreenName',
 'hits_appInfo.landingScreenName',
 'hits_appInfo.screenName',
 '

In [35]:
df['totals_transactions']

19208      0.0
686476     0.0
595207     0.0
417283     0.0
1559734    0.0
1366841    0.0
1579340    0.0
1293684    0.0
236476     0.0
919088     0.0
393061     0.0
140922     0.0
172617     0.0
1610196    0.0
1593026    0.0
1155363    0.0
1639744    0.0
1376115    0.0
1156739    0.0
269115     0.0
529653     0.0
97045      0.0
233118     0.0
1038186    0.0
994175     0.0
1017180    0.0
430726     0.0
1171173    0.0
6382       0.0
141421     0.0
          ... 
1702672    1.0
1702687    1.0
1702694    1.0
1702702    1.0
1702712    1.0
1702716    1.0
1702717    1.0
1702724    1.0
1702730    1.0
1702733    1.0
1702740    1.0
1705492    1.0
1705521    1.0
1705532    1.0
1705538    1.0
1705556    1.0
1705570    1.0
1705583    1.0
1705609    1.0
1705616    2.0
1707687    1.0
1707819    1.0
1707850    1.0
1707851    1.0
1707872    1.0
1707890    1.0
1707893    1.0
1707894    1.0
1707901    1.0
1707907    1.0
Name: totals_transactions, Length: 38514, dtype: float64

# CNN model and training process

In [46]:
import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import torch.utils.data as data_utils
# training_data = training_data.values.astype('float')
# training_data = torch.Tensor(training_data)
# target = torch.Tensor(target.values)

# loss_list = []
# epoch = 10
# training_data = training_data.unsqueeze(0)
# training_data = training_data.permute(1,0,2)
# target = target.unsqueeze(0)
# target = target.permute(1,0)
# train = data_utils.TensorDataset(training_data, target)


class CNN_net(nn.Module):
    def __init__(self,str_cols, num_cols):
        
        super(CNN_net, self).__init__()
        
        self.fc1 = nn.Sequential(

            nn.Conv1d(1, 8, kernel_size=3),
            nn.BatchNorm1d(8),
            nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
        self.fc2 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=3),
            nn.BatchNorm1d(16),
#             nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
#         self.fc3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear((len(str_cols) + len(num_cols) - 4 ) * 32,1)
        self.fc4 = nn.Linear(144,1)
#         self.fc5 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        out = x.reshape(x.size(0), -1)
#         print(out.size())
#         out = self.fc3(out)
        f = self.fc4(out)
#         f = self.fc5(f)
        return f


net = CNN_net(str_cols, num_cols)
net = net.cuda()
print(net )

#zero the gradient buffers
net.zero_grad()


epoch = 600

#create the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=31e-4)
criterion = nn.MSELoss()

for i in range(epoch):
#     if i%100 == 0:
    print(i)
    torch.cuda.empty_cache()
    running_loss = 0
    train_loader = data_utils.DataLoader(train, batch_size=256, shuffle=True)
    for trainx,labelx in train_loader:
        #print(trainx.size())
        optimizer.zero_grad()
        loss = criterion(net(trainx), labelx)
        loss_list.append(loss)
        loss.backward()
        optimizer.step() 
        running_loss += float(loss.item())
    print(running_loss)

#testing loop
# label = testing_data['totals_transactionRevenue']
# label = torch.Tensor(label.values)
# testing_data = testing_data[str_cols + num_cols]
# testing_data = torch.Tensor(testing_data.values)
# output = net(testing_data)
# print(output)

CNN_net(
  (fc1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (fc2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc4): Linear(in_features=144, out_features=1, bias=True)
)
0
8366.989524841309
1
2519.8633394241333
2
2229.5408220291138
3
2125.529867172241
4
2029.0451316833496
5
1980.8216037750244
6
1910.8253917694092
7
1905.7720022201538
8
1852.2175827026367
9
1816.0460319519043
10
1816.9835019111633
11
1806.731297492981
12
1811.3546676635742
13
1796.6388120651245
14
1779.9613065719604
15
1774.6698460578918
16
1772.120129108429
17
1760.2809886932373
18
1757.7813129425049
19
1753.015396118164
20
1742.796389579773
21
1734.4043827056885
22
1741.348

KeyboardInterrupt: ignored

In [14]:
output

tensor([[10.4859],
        [ 0.4510],
        [ 0.0597],
        ...,
        [13.0630],
        [15.6613],
        [15.2618]], device='cuda:0', grad_fn=<ThAddmmBackward>)

# initial loss

In [47]:
net.eval()
label = testing_data['totals_transactionRevenue']
label = torch.cuda.FloatTensor(label.values)
testing = testing_data[str_cols + num_cols]
testing = torch.cuda.FloatTensor(testing.values)


testing = testing.unsqueeze(0)
testing = testing.permute(1,0,2)
label = label.unsqueeze(0)
label = label.permute(1,0)
output = net(testing)
criterion = torch.nn.MSELoss()
loss = torch.sqrt(criterion(output, label))
loss

tensor(3.8621, device='cuda:0', grad_fn=<SqrtBackward>)

# calculate the final cumulative loss (loss of the the predition for user's whole spending)

In [12]:
import numpy as np
c = output.cpu().detach().numpy()
c = np.exp(c) -1
c1 = pandas.DataFrame(c)
c1.columns = ['pred']

cc = testing_id.to_frame().values
cc = pandas.DataFrame(cc)
cc.columns = ['id']

z = pandas.concat([c1,cc], axis=1, join='inner')
# x1 = z.groupby('id','pred').sum()
x1 = z.groupby(['id'])['pred'].agg('sum').reset_index()
x1['pred'] = x1['pred'] .apply(lambda x: math.log(1 + x))


c = label.cpu().detach().numpy()
c = np.exp(c) -1
c1 = pandas.DataFrame(c)
c1.columns = ['label']

cc = testing_id.to_frame().values
cc = pandas.DataFrame(cc)
cc.columns = ['id']

z = pandas.concat([c1,cc], axis=1, join='inner')

# x2 = z.groupby('id').agg(['sum'])
x2 = z.groupby(['id'])['label'].agg('sum').reset_index()
x2['label'] = x2['label'] .apply(lambda x: math.log(1 + x))
# x2['sum'] = s2['sum'].apply(lambda x: math.log(1 + x))
net.eval()
labels = x2['label']
labels = torch.cuda.FloatTensor(labels.values)
outs = x1['pred']
outs = torch.cuda.FloatTensor(outs.values)
criterion = torch.nn.MSELoss()
loss = torch.sqrt(criterion(outs, labels))



ValueError: ignored

In [0]:
loss

tensor(3.7009, device='cuda:0')